In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split

In [2]:
house_data = pd.read_csv('~/Downloads/train.csv')
house_data.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [3]:
columns = ['Id',	'OverallQual',	'YearBuilt',	'TotalBsmtSF',	'Electrical',	'GrLivArea',	'FullBath',	'GarageType',	'GarageCars',	'GarageArea',	'Fence',	'MiscFeature',	'SalePrice']
cls_house_data = house_data[columns]

In [4]:
cls_house_data.head(1)

,Id,OverallQual,YearBuilt,TotalBsmtSF,Electrical,GrLivArea,FullBath,GarageType,GarageCars,GarageArea,Fence,MiscFeature,SalePrice
0,1,7,2003,856,SBrkr,1710,2,Attchd,2,548,NaN,NaN,208500


In [5]:
cls_house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           1460 non-null   int64 
 1   OverallQual  1460 non-null   int64 
 2   YearBuilt    1460 non-null   int64 
 3   TotalBsmtSF  1460 non-null   int64 
 4   Electrical   1459 non-null   object
 5   GrLivArea    1460 non-null   int64 
 6   FullBath     1460 non-null   int64 
 7   GarageType   1379 non-null   object
 8   GarageCars   1460 non-null   int64 
 9   GarageArea   1460 non-null   int64 
 10  Fence        281 non-null    object
 11  MiscFeature  54 non-null     object
 12  SalePrice    1460 non-null   int64 
dtypes: int64(9), object(4)
memory usage: 148.4+ KB


In [6]:
cls_house_data.describe(include = 'all')

,Id,OverallQual,YearBuilt,TotalBsmtSF,Electrical,GrLivArea,FullBath,GarageType,GarageCars,GarageArea,Fence,MiscFeature,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1459,1460.000000,1460.000000,1379,1460.000000,1460.000000,281,54,1460.000000
unique,NaN,NaN,NaN,NaN,5,NaN,NaN,6,NaN,NaN,4,4,NaN
top,NaN,NaN,NaN,NaN,SBrkr,NaN,NaN,Attchd,NaN,NaN,MnPrv,Shed,NaN
freq,NaN,NaN,NaN,NaN,1334,NaN,NaN,870,NaN,NaN,157,49,NaN
mean,730.500000,6.099315,1971.267808,1057.429452,NaN,1515.463699,1.565068,NaN,1.767123,472.980137,NaN,NaN,180921.195890
std,421.610009,1.382997,30.202904,438.705324,NaN,525.480383,0.550916,NaN,0.747315,213.804841,NaN,NaN,79442.502883
min,1.000000,1.000000,1872.000000,0.000000,NaN,334.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,34900.000000
25%,365.750000,5.000000,1954.000000,795.750000,NaN,1129.500000,1.000000,NaN,1.000000,334.500000,NaN,NaN,129975.000000
50%,730.500000,6.000000,1973.000000,991.500000,NaN,1464.000000,2.000000,NaN,2.000000,480.000000,NaN,NaN,163000.000000
75%,1095.250000,7.000000,2000.000000,1298.250000,NaN,1776.750000,2.000000,NaN,2.000000,576.000000,NaN,NaN,214000.000000


In [7]:
# now as there is a large ammount of (i.e. more than 90 %) null or missing data in two of the columns namely 'Fence' and 'MiscFeature' and even though MiscFeature can be somewhat important the ammount of missing values renders it useless so we will simply drop it.
cls_house_data.drop(columns = ['Fence', 'MiscFeature'], inplace = True)

C:\Users\nothi\AppData\Local\Temp\ipykernel_8600\579115912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cls_house_data.drop(columns = ['Fence', 'MiscFeature'], inplace = True)


In [8]:
cls_house_data.info()
# now we have two catagorical columns which we will have to encode using suitable encoding methods and and two columns where few values are missing: 'Electrical' (1) and 'GarageType' (81)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           1460 non-null   int64 
 1   OverallQual  1460 non-null   int64 
 2   YearBuilt    1460 non-null   int64 
 3   TotalBsmtSF  1460 non-null   int64 
 4   Electrical   1459 non-null   object
 5   GrLivArea    1460 non-null   int64 
 6   FullBath     1460 non-null   int64 
 7   GarageType   1379 non-null   object
 8   GarageCars   1460 non-null   int64 
 9   GarageArea   1460 non-null   int64 
 10  SalePrice    1460 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 125.6+ KB


In [9]:
# Function to Impute GarageType

def setGarageType(qual, lowliv, upliv, lowbsmt, upbsmt, year_type, impute_value):
  '''This function is used by treat_GarageType function to impute the missing values using the value found by that function'''

  if year_type.lower() in ['more', 'greater']:
    cls_house_data.loc[
            (cls_house_data.OverallQual == qual) &
            (cls_house_data.GrLivArea > lowliv) &
            (cls_house_data.GrLivArea < upliv) &
            (cls_house_data.TotalBsmtSF > lowbsmt) &
            (cls_house_data.TotalBsmtSF < upbsmt) &
            (pd.isnull(cls_house_data.GarageType)), 'GarageType'] = impute_value
  else:
    cls_house_data.loc[
            (cls_house_data.OverallQual == qual) &
            (cls_house_data.GrLivArea > lowliv) &
            (cls_house_data.GrLivArea < upliv) &
            (cls_house_data.TotalBsmtSF > lowbsmt) &
            (cls_house_data.TotalBsmtSF < upbsmt) &
            (cls_house_data.YearBuilt < 1955) &
            (pd.isnull(cls_house_data.GarageType)), 'GarageType'] = impute_value

def treat_GarageType(qual, lowliv, upliv, lowbsmt, upbsmt, year_type):
  '''It takes OverallQual, Lower-GrLivArea, Upper-GrLivArea, Lower-TotalBsmtSF, Upper-TotalBsmtSF as Parameters where,
  Lower-GrLivArea: Lower Range from which the GrLivArea should be greater
  Upper-GrLivArea: Upper Range from which the GrLivArea should be lower
  Lower-TotalBsmtSF: Lower Range from which the TotalBsmtSF should be greater
  Upper-TotalBsmtSF: Upper Range from which the TotalBsmtSF should be lower

  and then get the most frequent "GarageType" value based on these conditions and used it to impute the missing GarageType values'''

  if year_type.lower() in ['more', 'greater']:
    fil_data = cls_house_data.loc[
            (cls_house_data.OverallQual == qual) &
            (cls_house_data.GrLivArea > lowliv) &
            (cls_house_data.GrLivArea < upliv) &
            (cls_house_data.TotalBsmtSF > lowbsmt) &
            (cls_house_data.TotalBsmtSF < upbsmt) &
            (cls_house_data.YearBuilt > 1955)]
  else:
    fil_data = cls_house_data.loc[
            (cls_house_data.OverallQual == qual) &
            (cls_house_data.GrLivArea > lowliv) &
            (cls_house_data.GrLivArea < upliv) &
            (cls_house_data.TotalBsmtSF > lowbsmt) &
            (cls_house_data.TotalBsmtSF < upbsmt) &
            (cls_house_data.YearBuilt < 1955)]

  if fil_data.empty:
    return
  else:
    gtype = fil_data['GarageType'].value_counts().idxmax()
    setGarageType(qual, lowliv, upliv, lowbsmt, upbsmt, year_type, gtype)

In [10]:
# Missing values

# Electrical ---------------------------------------------------------------------

print(cls_house_data['Electrical'].value_counts())
print()
print(cls_house_data.loc[pd.isnull(cls_house_data.Electrical)])
print()
print(cls_house_data.loc[cls_house_data.OverallQual == 10, 'Electrical'].value_counts())
print()
# all but houses with quality 1 and 2 have 'SBrkr' as most frequent value so a our missing data-point have quality of 5 we will use 'SBrkr'

cls_house_data.loc[pd.isnull(cls_house_data.Electrical), 'Electrical'] = 'SBrkr'
cls_house_data.loc[cls_house_data.Id == 1380]

# GarageType ---------------------------------------------------------------------
print(cls_house_data.loc[pd.isnull(cls_house_data.GarageType)]['OverallQual'].value_counts()) # as we can see most of houses are 4, 5 and 6 quality so we will treat them
# Quality: 6---------------------------------------------
treat_GarageType(6, 500, 1000, 500, 1000, 'more')
treat_GarageType(6, 1000, 1500, 500, 1000, 'more')
treat_GarageType(6, 1500, 3500, 500, 1000, 'more')

treat_GarageType(6, 500, 1000, 1000, 2000, 'more')
treat_GarageType(6, 1000, 1500, 1000, 2000, 'more')
treat_GarageType(6, 1500, 3500, 1000, 2000, 'more')

treat_GarageType(6, 500, 1000, 500, 1000, 'less')
treat_GarageType(6, 1000, 1500, 500, 1000, 'less')
treat_GarageType(6, 1500, 3500, 500, 1000, 'less')

treat_GarageType(6, 500, 1000, 1000, 2000, 'less')
treat_GarageType(6, 1000, 1500, 1000, 2000, 'less')
treat_GarageType(6, 1500, 3500, 1000, 2000, 'less')


# Quality: 5---------------------------------------------
treat_GarageType(5, 500, 1000, 500, 1000, 'more')
treat_GarageType(5, 1000, 1500, 500, 1000, 'more')
treat_GarageType(5, 1500, 3500, 500, 1000, 'more')

treat_GarageType(5, 500, 1000, 1000, 2000, 'more')
treat_GarageType(5, 1000, 1500, 1000, 2000, 'more')
treat_GarageType(5, 1500, 3500, 1000, 2000, 'more')

treat_GarageType(5, 500, 1000, 500, 1000, 'less')
treat_GarageType(5, 1000, 1500, 500, 1000, 'less')
treat_GarageType(5, 1500, 3500, 500, 1000, 'less')

treat_GarageType(5, 500, 1000, 1000, 2000, 'less')
treat_GarageType(5, 1000, 1500, 1000, 2000, 'less')
treat_GarageType(5, 1500, 3500, 1000, 2000, 'less')


# Quality: 4---------------------------------------------
treat_GarageType(4, 500, 1000, 500, 1000, 'more')
treat_GarageType(4, 1000, 1500, 500, 1000, 'more')
treat_GarageType(4, 1500, 3500, 500, 1000, 'more')

treat_GarageType(4, 500, 1000, 1000, 2000, 'more')
treat_GarageType(4, 1000, 1500, 1000, 2000, 'more')
treat_GarageType(4, 1500, 3500, 1000, 2000, 'more')

treat_GarageType(4, 500, 1000, 500, 1000, 'less')
treat_GarageType(4, 1000, 1500, 500, 1000, 'less')
treat_GarageType(4, 1500, 3500, 500, 1000, 'less')

treat_GarageType(4, 500, 1000, 1000, 2000, 'less')
treat_GarageType(4, 1000, 1500, 1000, 2000, 'less')
treat_GarageType(4, 1500, 3500, 1000, 2000, 'less')

cls_house_data.dropna(subset = 'GarageType', inplace = True) # there are still 20 data-points where GarageType value is missing so i just dropped them as 20 points arn't that high of count


# GarageCars ---------------------------------------------------------------------
# now even though it son't have any missing values it does have a lots of zeros wherever there was missing GarageType so now we will handle those by using 'YearBuilt' and 'GrLivArea'
print(cls_house_data.loc[cls_house_data.GarageCars == 2]) # as we can see 'GarageCars' have '2' mostly when year is greater than 1975 and 'GrLivArea' is grater than 1200
cls_house_data.loc[(cls_house_data.GarageCars == 0) & (cls_house_data.GrLivArea > 1200) & (cls_house_data.YearBuilt > 1975), 'GarageCars'] = 2
cls_house_data.loc[(cls_house_data.GarageCars == 0) & (cls_house_data.GrLivArea < 1200) & (cls_house_data.YearBuilt < 1975), 'GarageCars'] = 1
cls_house_data.loc[cls_house_data.GarageCars == 0]#.count()
cls_house_data.loc[(cls_house_data.YearBuilt < 1975) & (cls_house_data.GrLivArea > 1700) & (cls_house_data.TotalBsmtSF > 800) & (cls_house_data.GarageCars==0), 'GarageCars'] = 2
cls_house_data.loc[(cls_house_data.YearBuilt < 1975) & (cls_house_data.GrLivArea > 1200) & (cls_house_data.TotalBsmtSF > 700) & (cls_house_data.GarageCars==0), 'GarageCars'] = 1
cls_house_data.loc[(cls_house_data.YearBuilt > 1975) & (cls_house_data.GrLivArea > 1200) & (cls_house_data.TotalBsmtSF > 700) & (cls_house_data.GarageCars==0), 'GarageCars'] = 2


# GarageArea ---------------------------------------------------------------------
# same as GarageCars and will use 'GarageCars'
cls_house_data.loc[(cls_house_data.GarageCars == 1) & (cls_house_data.GarageArea == 0), 'GarageArea'] = cls_house_data.loc[cls_house_data.GarageCars == 1, 'GarageArea'].mean()
cls_house_data.loc[(cls_house_data.GarageCars == 2) & (cls_house_data.GarageArea == 0), 'GarageArea'] = cls_house_data.loc[cls_house_data.GarageCars == 2, 'GarageArea'].mean()

# --------------------------------------------------------------------------------
print(cls_house_data.loc[cls_house_data.GarageCars == 0].count()) # as we can see there re still 14 datapoint where value is 0 we will just drop them
cls_house_data.drop(cls_house_data[cls_house_data.GarageCars == 0].index, inplace=True)
cls_house_data.loc[cls_house_data.GarageCars == 0].count()

Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

        Id  OverallQual  YearBuilt  TotalBsmtSF Electrical  GrLivArea  \
1379  1380            5       2006          384        NaN       1394   

      FullBath GarageType  GarageCars  GarageArea  SalePrice  
1379         2    BuiltIn           2         400     167500  

Electrical
SBrkr    18
Name: count, dtype: int64

OverallQual
5    32
4    26
6    12
3     6
1     2
7     1
2     1
8     1
Name: count, dtype: int64
        Id  OverallQual  YearBuilt  TotalBsmtSF Electrical  GrLivArea  \
0        1            7       2003          856      SBrkr       1710   
1        2            6       1976         1262      SBrkr       1262   
2        3            7       2001          920      SBrkr       1786   
5        6            5       1993          796      SBrkr       1362   
6        7            8       2004         1686      SBrkr       1694   
...    ...          ...     

C:\Users\nothi\AppData\Local\Temp\ipykernel_8600\1408662810.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cls_house_data.dropna(subset = 'GarageType', inplace = True) # there are still 20 data-points where GarageType value is missing so i just dropped them as 20 points arn't that high of count
C:\Users\nothi\AppData\Local\Temp\ipykernel_8600\1408662810.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '272.4594594594595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cls_house_data.loc[(cls_house_data.GarageCars == 1) & (cls_house_data.GarageArea == 0), 'GarageArea'] = cls_house_data.loc[cls_house_data.GarageCars == 1, 'GarageArea'].mean()
C:\Users\nothi\AppData\Local\Te

Id             0
OverallQual    0
YearBuilt      0
TotalBsmtSF    0
Electrical     0
GrLivArea      0
FullBath       0
GarageType     0
GarageCars     0
GarageArea     0
SalePrice      0
dtype: int64

In [11]:
cls_house_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1426 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           1426 non-null   int64  
 1   OverallQual  1426 non-null   int64  
 2   YearBuilt    1426 non-null   int64  
 3   TotalBsmtSF  1426 non-null   int64  
 4   Electrical   1426 non-null   object 
 5   GrLivArea    1426 non-null   int64  
 6   FullBath     1426 non-null   int64  
 7   GarageType   1426 non-null   object 
 8   GarageCars   1426 non-null   int64  
 9   GarageArea   1426 non-null   float64
 10  SalePrice    1426 non-null   int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 133.7+ KB


In [12]:
# Scaling

scaled_data = cls_house_data.copy()
num_columns = scaled_data.select_dtypes(include = ['int', 'float']).columns
print(num_columns)
rs = RobustScaler()
for col in num_columns:
  scaled_data[col] = rs.fit_transform(scaled_data[[col]])

scaled_data

Index(['Id', 'OverallQual', 'YearBuilt', 'TotalBsmtSF', 'GrLivArea',
       'FullBath', 'GarageCars', 'GarageArea', 'SalePrice'],
      dtype='object')


,Id,OverallQual,YearBuilt,TotalBsmtSF,Electrical,GrLivArea,FullBath,GarageType,GarageCars,GarageArea,SalePrice
0,-1.000000,0.5,0.638298,-0.278656,SBrkr,0.367932,0.0,Attchd,0.0,0.296380,0.521739
1,-0.998629,0.0,0.063830,0.523715,SBrkr,-0.326104,0.0,Attchd,0.0,-0.101810,0.197901
2,-0.997258,0.5,0.595745,-0.152174,SBrkr,0.485670,0.0,Attchd,0.0,0.567873,0.701649
3,-0.995888,0.5,-1.234043,-0.476285,SBrkr,0.378776,-1.0,Detchd,1.0,0.721719,-0.299850
4,-0.994517,1.0,0.574468,0.292490,SBrkr,1.123935,0.0,Attchd,1.0,1.599548,1.019490
...,...,...,...,...,...,...,...,...,...,...,...
1455,0.994517,0.0,0.553191,-0.086957,SBrkr,0.270333,0.0,Attchd,0.0,-0.101810,0.119940
1456,0.995888,0.0,0.106383,1.077075,SBrkr,0.930287,0.0,Attchd,0.0,0.079186,0.539730
1457,0.997258,0.5,-0.680851,0.306324,SBrkr,1.343919,0.0,Attchd,-1.0,-1.042986,1.217391
1458,0.998629,-0.5,-0.489362,0.160079,FuseA,-0.611154,-1.0,Attchd,-1.0,-1.097285,-0.274363


In [13]:
# Encoding
# we have two catagorical columns: 'Electrical' and 'GarageType'

print(scaled_data.GarageType.unique(),scaled_data.Electrical.unique())
# as there are only 11 total label (i.e. 6 in 'GarageType' and 5 in 'Electrical') we will simply use one-hot encoding but can also use 'Target Encoding' where we group our target variable by thses unique labels and get a mean label wise which we then use to encode labels

scaled_data = pd.get_dummies(scaled_data, columns = ['Electrical', 'GarageType'], drop_first = True)

['Attchd' 'Detchd' 'BuiltIn' 'CarPort' 'Basment' '2Types'] ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix']


In [14]:
scaled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1426 entries, 0 to 1459
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1426 non-null   float64
 1   OverallQual         1426 non-null   float64
 2   YearBuilt           1426 non-null   float64
 3   TotalBsmtSF         1426 non-null   float64
 4   GrLivArea           1426 non-null   float64
 5   FullBath            1426 non-null   float64
 6   GarageCars          1426 non-null   float64
 7   GarageArea          1426 non-null   float64
 8   SalePrice           1426 non-null   float64
 9   Electrical_FuseF    1426 non-null   bool   
 10  Electrical_FuseP    1426 non-null   bool   
 11  Electrical_Mix      1426 non-null   bool   
 12  Electrical_SBrkr    1426 non-null   bool   
 13  GarageType_Attchd   1426 non-null   bool   
 14  GarageType_Basment  1426 non-null   bool   
 15  GarageType_BuiltIn  1426 non-null   bool   
 16  GarageType_

In [15]:
# Transformation

print(scaled_data.skew())
tr = PowerTransformer(method = 'yeo-johnson')

scaled_data[['SalePrice', 'TotalBsmtSF', 'GrLivArea', 'GarageArea']] = tr.fit_transform(scaled_data[['SalePrice', 'TotalBsmtSF', 'GrLivArea', 'GarageArea']])

print(scaled_data.skew())
final_data = scaled_data.copy()

Id                     0.003520
OverallQual            0.314104
YearBuilt             -0.623062
TotalBsmtSF            1.630502
GrLivArea              1.376118
FullBath               0.020733
GarageCars             0.220198
GarageArea             0.824495
SalePrice              1.914257
Electrical_FuseF       7.360148
Electrical_FuseP      26.673918
Electrical_Mix        37.762415
Electrical_SBrkr      -3.049093
GarageType_Attchd     -0.562962
GarageType_Basment     8.498119
GarageType_BuiltIn     3.646681
GarageType_CarPort    12.481124
GarageType_Detchd      0.985961
dtype: float64
Id                     0.003520
OverallQual            0.314104
YearBuilt             -0.623062
TotalBsmtSF           -0.053366
GrLivArea              0.042397
FullBath               0.020733
GarageCars             0.220198
GarageArea             0.053576
SalePrice             -0.021893
Electrical_FuseF       7.360148
Electrical_FuseP      26.673918
Electrical_Mix        37.762415
Electrical_SBrkr      -3.

In [16]:
# Splitting the data

x = final_data.drop('SalePrice', axis = 1)
y = final_data.SalePrice

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1069, 17), (1069,), (357, 17), (357,))